In [1]:
from pyowm import OWM
from pyowm.weatherapi25.forecast import Forecast
from config import OWM_API_key as key, port, host
from pprint import pprint
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure, InvalidDocument, DuplicateKeyError
# import json
import os
from os import path


API_key = key
host = host
port = port
filename = 'resources/zip_list.csv'
global zipcode
owm = OWM(API_key) # the OWM object
global obs
global reception_time
global zid# = Zip_Location.get_ID()
global zlon# = location.get_lon()
global zlat# = location.get_lat()
global client

def set_location(code):
    ''' Get the latitude and longitude corrosponding to the zip code.
        
        :param code: the zip code to find weather data about
        :type code: string
    '''
    global obs, zlat, zlon
    obs = owm.weather_at_zip_code(f'{code}', 'us')
    location = obs.get_location()
    zlon = location.get_lon()
    zlat = location.get_lat()
    return


def current():
    ''' Dump the current weather to a json

        :return current: the currently observed weather data
        :type current: dict
    '''
    global obs
    current = obs.to_JSON() # dump all that weather data to a json object
    return(current)


def five_day():
    ''' Get each weather forecast for the corrosponding zip code. 
    
        :return five_day: the five day, every three hours, forecast for the zip code\
        :type five_day: dict
    '''
    global obs, zlat, zlon
    forecaster = owm.three_hours_forecast_at_coords(zlat, zlon)
    forecast = forecaster.get_forecast()
    five_day = forecast.to_JSON()
    return(five_day)


# YOU DON'T HAVE THE API ACCESS TO GET THIS TYPE OF FORECAST REQUEST
# def daily():
#     ''' Fetch the daily forecast. '''
#     forecaster = owm.daily_forecast_at_coords(zlat, zlon)
#     forecast = forecaster.get_forecast()
#     daily = forecast.to_JSON()
#     return(daily)


def check_db_access(host, port):
    ''' A check that there is write access to the database
    
        :param host: the database host
        :type host: string
        :param port: the database connection port
        :type port: int
    '''
    client = MongoClient(host=host, port=port)
    try:
        # The ismaster command is cheap and does not require auth.
        client.admin.command('ismaster')
        print('Connection made')
    except ConnectionFailure:
        print("Server not available")
        return

    # check the database connections
        # Get a count of the number of databases at the connection (accessible through that port)
        # before attempting to add to it
    db_count_pre = len(client.list_database_names())
        # Add a database and collection
    db = client.test_db
    col = db.test_col

    # Insert something to the db
    post = {'name':'Chuck VanHoff',
           'age':'38',
           'hobby':'gardening'
           }
    col.insert_one(post)

        # Get a count of the databases after adding one
    db_count_post = len(client.list_database_names())

    if db_count_pre-db_count_post>=0:
        print('Your conneciton is flipped up')
    else:
        print('You have write access')

    client.drop_database(db)
    return(client)

def to_json(data, code):
    ''' Store the collected data as a json file in the case that the database
        is not connected or otherwise unavailable.
        
        :param data: the dictionary created from the api calls
        :type data: dict
        :param code: the zip code associated with the data from the list codes
        :type code: sting
    '''
    collection = data
    directory = os.getcwd()
    save_path = path.join(directory, 'Data', f'{code}.json')
    Data = open(save_path, 'a+')
    Data.write(collection)
    Data.close()
    return

def load(data, client):
    ''' Load the data to the database if possible, otherwise write to json file. 
        
        :param data: the dictionary created from the api calls
        :type data: dict
        :param client: the pymongo client object
        :type client: MongoClient
    '''
    if type(data) == dict:
        try:
            db = client.forcast
            col = db.code
            col.insert_one(data)
        except DuplicateKeyError:
            client.close()
            print('closed db connection')
            to_json(data, code)
            print('Wrote to json')
    else:
        print('data type is not dict')
        client.close()
        print('closed db connection')
    return

def write_list_to_file(zip_list, filename):
    """ Write the zip codes to csv file.
        
        :param zip_list: the list created from the zip codes dataframe
        :type zip_list: list stings
        :param filename: the name of the file
        :type filename: sting
    """
    with open(filename, "w") as z_list:
        z_list.write(z for z in zip_list)
    return
        
def read_list_from_file(filename):
    """ Read the zip codes list from the csv file.
        
        :param filename: the name of the file
        :type filename: sting
    """
    with open(filename, "r") as z_list:
        return z_list.read().strip().split(',')

def get_weather(codes):
    ''' Get the weather from the API and load it to the database. 
    
    :param codes: list of zip codes
    :type codes: list of strings
    '''
    client = check_db_access(host, port)
    for code in codes:
        data = {}
        set_location(code)
        data.update({'zipcode': code,
                     'current': current(),
                     'five_day': five_day(),
                    })
        load(data, client)
    client.close()
    return

In [2]:
def make_zip_list():
    ''' Make a list of zip codes in the specified state.
        
        :param state: the two-letter abreviation of the state whose zip codes you'd like listed
        :type state: string
    '''
    import pandas as pd
    
    df = pd.read_csv("resources/zip_code_database.csv")
    # Make a datafram from the rows with the specified state and write the 'zip' column to a list 
#     zip_list = df.loc[lambda df: df['state'] == f'{state.upper()}']['zip'].tolist()
    zip_list = df['zip'].tolist()
    return(zip_list)

In [4]:
from pyowm.exceptions.api_response_error import NotFoundError

codes = make_zip_list() #read_list_from_file(filename)
n=0
successes=0
exceptions=0
zip_list = []

for code in codes:
#     data = {}
    if n<5:
        zipp = f'{code}'
#         print(zipp)
        zip_list.append(f'{code}')
        print(f'appended {code} to zip_list')
    else:
        print(f'n is greater than 5')
        break
#     try:
#         print('try setting loation for ', zip_list[n])
#         set_location(code)
#         print(f'successfully set location for {code}')
#         successes+=1
# #         write_list_to_file(zip_list, 'resources/success_zips.csv')
#     except NotFoundError:
#         print("except", f'NotFoundError with zipcode {code}')
# #         write_list_to_file(zip_list, 'resources/exception_zips.csv')
#         exceptions+=1
#         pass
# #     n+=1
# print(f'successes = {successes}; exceptions = {exceptions}')
# get_weather(codes)

appended 501 to zip_list
appended 544 to zip_list
appended 601 to zip_list
appended 602 to zip_list
appended 603 to zip_list
appended 604 to zip_list
appended 605 to zip_list
appended 606 to zip_list
appended 610 to zip_list
appended 611 to zip_list
appended 612 to zip_list
appended 613 to zip_list
appended 614 to zip_list
appended 616 to zip_list
appended 617 to zip_list
appended 622 to zip_list
appended 623 to zip_list
appended 624 to zip_list
appended 627 to zip_list
appended 631 to zip_list
appended 636 to zip_list
appended 637 to zip_list
appended 638 to zip_list
appended 641 to zip_list
appended 646 to zip_list
appended 647 to zip_list
appended 650 to zip_list
appended 652 to zip_list
appended 653 to zip_list
appended 656 to zip_list
appended 659 to zip_list
appended 660 to zip_list
appended 662 to zip_list
appended 664 to zip_list
appended 667 to zip_list
appended 669 to zip_list
appended 670 to zip_list
appended 674 to zip_list
appended 676 to zip_list
appended 677 to zip_list


appended 4412 to zip_list
appended 4413 to zip_list
appended 4414 to zip_list
appended 4415 to zip_list
appended 4416 to zip_list
appended 4417 to zip_list
appended 4418 to zip_list
appended 4419 to zip_list
appended 4420 to zip_list
appended 4421 to zip_list
appended 4422 to zip_list
appended 4424 to zip_list
appended 4426 to zip_list
appended 4427 to zip_list
appended 4428 to zip_list
appended 4429 to zip_list
appended 4430 to zip_list
appended 4431 to zip_list
appended 4434 to zip_list
appended 4435 to zip_list
appended 4438 to zip_list
appended 4441 to zip_list
appended 4442 to zip_list
appended 4443 to zip_list
appended 4444 to zip_list
appended 4448 to zip_list
appended 4449 to zip_list
appended 4450 to zip_list
appended 4451 to zip_list
appended 4453 to zip_list
appended 4454 to zip_list
appended 4455 to zip_list
appended 4456 to zip_list
appended 4457 to zip_list
appended 4459 to zip_list
appended 4460 to zip_list
appended 4461 to zip_list
appended 4462 to zip_list
appended 446

appended 6852 to zip_list
appended 6853 to zip_list
appended 6854 to zip_list
appended 6855 to zip_list
appended 6856 to zip_list
appended 6857 to zip_list
appended 6858 to zip_list
appended 6859 to zip_list
appended 6860 to zip_list
appended 6870 to zip_list
appended 6875 to zip_list
appended 6876 to zip_list
appended 6877 to zip_list
appended 6878 to zip_list
appended 6879 to zip_list
appended 6880 to zip_list
appended 6881 to zip_list
appended 6883 to zip_list
appended 6888 to zip_list
appended 6889 to zip_list
appended 6890 to zip_list
appended 6896 to zip_list
appended 6897 to zip_list
appended 6901 to zip_list
appended 6902 to zip_list
appended 6903 to zip_list
appended 6904 to zip_list
appended 6905 to zip_list
appended 6906 to zip_list
appended 6907 to zip_list
appended 6910 to zip_list
appended 6911 to zip_list
appended 6912 to zip_list
appended 6913 to zip_list
appended 6914 to zip_list
appended 6920 to zip_list
appended 6921 to zip_list
appended 6922 to zip_list
appended 692

appended 10588 to zip_list
appended 10589 to zip_list
appended 10590 to zip_list
appended 10591 to zip_list
appended 10594 to zip_list
appended 10595 to zip_list
appended 10596 to zip_list
appended 10597 to zip_list
appended 10598 to zip_list
appended 10601 to zip_list
appended 10602 to zip_list
appended 10603 to zip_list
appended 10604 to zip_list
appended 10605 to zip_list
appended 10606 to zip_list
appended 10607 to zip_list
appended 10610 to zip_list
appended 10701 to zip_list
appended 10702 to zip_list
appended 10703 to zip_list
appended 10704 to zip_list
appended 10705 to zip_list
appended 10706 to zip_list
appended 10707 to zip_list
appended 10708 to zip_list
appended 10709 to zip_list
appended 10710 to zip_list
appended 10801 to zip_list
appended 10802 to zip_list
appended 10803 to zip_list
appended 10804 to zip_list
appended 10805 to zip_list
appended 10901 to zip_list
appended 10910 to zip_list
appended 10911 to zip_list
appended 10912 to zip_list
appended 10913 to zip_list
a

appended 12978 to zip_list
appended 12979 to zip_list
appended 12980 to zip_list
appended 12981 to zip_list
appended 12983 to zip_list
appended 12985 to zip_list
appended 12986 to zip_list
appended 12987 to zip_list
appended 12989 to zip_list
appended 12992 to zip_list
appended 12993 to zip_list
appended 12995 to zip_list
appended 12996 to zip_list
appended 12997 to zip_list
appended 12998 to zip_list
appended 13020 to zip_list
appended 13021 to zip_list
appended 13022 to zip_list
appended 13024 to zip_list
appended 13026 to zip_list
appended 13027 to zip_list
appended 13028 to zip_list
appended 13029 to zip_list
appended 13030 to zip_list
appended 13031 to zip_list
appended 13032 to zip_list
appended 13033 to zip_list
appended 13034 to zip_list
appended 13035 to zip_list
appended 13036 to zip_list
appended 13037 to zip_list
appended 13039 to zip_list
appended 13040 to zip_list
appended 13041 to zip_list
appended 13042 to zip_list
appended 13043 to zip_list
appended 13044 to zip_list
a

...

appended 91616 to zip_list
appended 91617 to zip_list
appended 91618 to zip_list
appended 91701 to zip_list
appended 91702 to zip_list
appended 91706 to zip_list
appended 91708 to zip_list
appended 91709 to zip_list
appended 91710 to zip_list
appended 91711 to zip_list
appended 91714 to zip_list
appended 91715 to zip_list
appended 91716 to zip_list
appended 91722 to zip_list
appended 91723 to zip_list
appended 91724 to zip_list
appended 91729 to zip_list
appended 91730 to zip_list
appended 91731 to zip_list
appended 91732 to zip_list
appended 91733 to zip_list
appended 91734 to zip_list
appended 91735 to zip_list
appended 91737 to zip_list
appended 91739 to zip_list
appended 91740 to zip_list
appended 91741 to zip_list
appended 91743 to zip_list
appended 91744 to zip_list
appended 91745 to zip_list
appended 91746 to zip_list
appended 91747 to zip_list
appended 91748 to zip_list
appended 91749 to zip_list
appended 91750 to zip_list
appended 91752 to zip_list
appended 91754 to zip_list
a

appended 94043 to zip_list
appended 94044 to zip_list
appended 94060 to zip_list
appended 94061 to zip_list
appended 94062 to zip_list
appended 94063 to zip_list
appended 94064 to zip_list
appended 94065 to zip_list
appended 94066 to zip_list
appended 94070 to zip_list
appended 94074 to zip_list
appended 94080 to zip_list
appended 94083 to zip_list
appended 94085 to zip_list
appended 94086 to zip_list
appended 94087 to zip_list
appended 94088 to zip_list
appended 94089 to zip_list
appended 94101 to zip_list
appended 94102 to zip_list
appended 94103 to zip_list
appended 94104 to zip_list
appended 94105 to zip_list
appended 94106 to zip_list
appended 94107 to zip_list
appended 94108 to zip_list
appended 94109 to zip_list
appended 94110 to zip_list
appended 94111 to zip_list
appended 94112 to zip_list
appended 94114 to zip_list
appended 94115 to zip_list
appended 94116 to zip_list
appended 94117 to zip_list
appended 94118 to zip_list
appended 94119 to zip_list
appended 94120 to zip_list
a

appended 95843 to zip_list
appended 95851 to zip_list
appended 95852 to zip_list
appended 95853 to zip_list
appended 95860 to zip_list
appended 95864 to zip_list
appended 95865 to zip_list
appended 95866 to zip_list
appended 95867 to zip_list
appended 95887 to zip_list
appended 95894 to zip_list
appended 95899 to zip_list
appended 95901 to zip_list
appended 95903 to zip_list
appended 95910 to zip_list
appended 95912 to zip_list
appended 95913 to zip_list
appended 95914 to zip_list
appended 95915 to zip_list
appended 95916 to zip_list
appended 95917 to zip_list
appended 95918 to zip_list
appended 95919 to zip_list
appended 95920 to zip_list
appended 95922 to zip_list
appended 95923 to zip_list
appended 95924 to zip_list
appended 95925 to zip_list
appended 95926 to zip_list
appended 95927 to zip_list
appended 95928 to zip_list
appended 95929 to zip_list
appended 95930 to zip_list
appended 95932 to zip_list
appended 95934 to zip_list
appended 95935 to zip_list
appended 95936 to zip_list
a

appended 98029 to zip_list
appended 98030 to zip_list
appended 98031 to zip_list
appended 98032 to zip_list
appended 98033 to zip_list
appended 98034 to zip_list
appended 98035 to zip_list
appended 98036 to zip_list
appended 98037 to zip_list
appended 98038 to zip_list
appended 98039 to zip_list
appended 98040 to zip_list
appended 98041 to zip_list
appended 98042 to zip_list
appended 98043 to zip_list
appended 98045 to zip_list
appended 98046 to zip_list
appended 98047 to zip_list
appended 98050 to zip_list
appended 98051 to zip_list
appended 98052 to zip_list
appended 98053 to zip_list
appended 98054 to zip_list
appended 98055 to zip_list
appended 98056 to zip_list
appended 98057 to zip_list
appended 98058 to zip_list
appended 98059 to zip_list
appended 98061 to zip_list
appended 98062 to zip_list
appended 98063 to zip_list
appended 98064 to zip_list
appended 98065 to zip_list
appended 98068 to zip_list
appended 98070 to zip_list
appended 98071 to zip_list
appended 98072 to zip_list
a

appended 99759 to zip_list
appended 99760 to zip_list
appended 99761 to zip_list
appended 99762 to zip_list
appended 99763 to zip_list
appended 99764 to zip_list
appended 99765 to zip_list
appended 99766 to zip_list
appended 99767 to zip_list
appended 99768 to zip_list
appended 99769 to zip_list
appended 99770 to zip_list
appended 99771 to zip_list
appended 99772 to zip_list
appended 99773 to zip_list
appended 99774 to zip_list
appended 99775 to zip_list
appended 99776 to zip_list
appended 99777 to zip_list
appended 99778 to zip_list
appended 99779 to zip_list
appended 99780 to zip_list
appended 99781 to zip_list
appended 99782 to zip_list
appended 99783 to zip_list
appended 99784 to zip_list
appended 99785 to zip_list
appended 99786 to zip_list
appended 99788 to zip_list
appended 99789 to zip_list
appended 99790 to zip_list
appended 99791 to zip_list
appended 99801 to zip_list
appended 99802 to zip_list
appended 99803 to zip_list
appended 99811 to zip_list
appended 99812 to zip_list
a

In [30]:
n=0
for n in range(len(zip_list)):
    try:
        print('try', zip_list[n])
        set_location(zip_list[n])
        print(f'successfully set location for {zip_list[n]}')
        successes+=1
#         write_list_to_file(zip_list, 'resources/success_zips.csv')
    except NotFoundError:
        print("except", f'NotFoundError with zipcode {code}')
#         write_list_to_file(zip_list, 'resources/exception_zips.csv')
        exceptions+=1
        pass

try 99950
successfully set location for 99950


In [34]:
for code in codes:
#     data = {}
    if n<5:
        zipp = f'{code}'
#         print(zipp)
        zip_list.append(f'{code}')
        print(f'appended {code} to zip_list')
    else:
        print(f'n is greater than 5')
        break

appended 501 to zip_list
appended 544 to zip_list
appended 601 to zip_list
appended 602 to zip_list
appended 603 to zip_list
appended 604 to zip_list
appended 605 to zip_list
appended 606 to zip_list
appended 610 to zip_list
appended 611 to zip_list
appended 612 to zip_list
appended 613 to zip_list
appended 614 to zip_list
appended 616 to zip_list
appended 617 to zip_list
appended 622 to zip_list
appended 623 to zip_list
appended 624 to zip_list
appended 627 to zip_list
appended 631 to zip_list
appended 636 to zip_list
appended 637 to zip_list
appended 638 to zip_list
appended 641 to zip_list
appended 646 to zip_list
appended 647 to zip_list
appended 650 to zip_list
appended 652 to zip_list
appended 653 to zip_list
appended 656 to zip_list
appended 659 to zip_list
appended 660 to zip_list
appended 662 to zip_list
appended 664 to zip_list
appended 667 to zip_list
appended 669 to zip_list
appended 670 to zip_list
appended 674 to zip_list
appended 676 to zip_list
appended 677 to zip_list


appended 4738 to zip_list
appended 4739 to zip_list
appended 4740 to zip_list
appended 4741 to zip_list
appended 4742 to zip_list
appended 4743 to zip_list
appended 4744 to zip_list
appended 4745 to zip_list
appended 4746 to zip_list
appended 4747 to zip_list
appended 4750 to zip_list
appended 4751 to zip_list
appended 4756 to zip_list
appended 4757 to zip_list
appended 4758 to zip_list
appended 4760 to zip_list
appended 4761 to zip_list
appended 4762 to zip_list
appended 4763 to zip_list
appended 4764 to zip_list
appended 4765 to zip_list
appended 4766 to zip_list
appended 4768 to zip_list
appended 4769 to zip_list
appended 4772 to zip_list
appended 4773 to zip_list
appended 4774 to zip_list
appended 4775 to zip_list
appended 4776 to zip_list
appended 4777 to zip_list
appended 4779 to zip_list
appended 4780 to zip_list
appended 4781 to zip_list
appended 4783 to zip_list
appended 4785 to zip_list
appended 4786 to zip_list
appended 4787 to zip_list
appended 4841 to zip_list
appended 484

appended 7303 to zip_list
appended 7304 to zip_list
appended 7305 to zip_list
appended 7306 to zip_list
appended 7307 to zip_list
appended 7308 to zip_list
appended 7309 to zip_list
appended 7310 to zip_list
appended 7311 to zip_list
appended 7395 to zip_list
appended 7399 to zip_list
appended 7401 to zip_list
appended 7403 to zip_list
appended 7405 to zip_list
appended 7407 to zip_list
appended 7410 to zip_list
appended 7416 to zip_list
appended 7417 to zip_list
appended 7418 to zip_list
appended 7419 to zip_list
appended 7420 to zip_list
appended 7421 to zip_list
appended 7422 to zip_list
appended 7423 to zip_list
appended 7424 to zip_list
appended 7428 to zip_list
appended 7430 to zip_list
appended 7432 to zip_list
appended 7435 to zip_list
appended 7436 to zip_list
appended 7438 to zip_list
appended 7439 to zip_list
appended 7440 to zip_list
appended 7442 to zip_list
appended 7444 to zip_list
appended 7446 to zip_list
appended 7450 to zip_list
appended 7451 to zip_list
appended 745

appended 10125 to zip_list
appended 10126 to zip_list
appended 10128 to zip_list
appended 10129 to zip_list
appended 10130 to zip_list
appended 10131 to zip_list
appended 10132 to zip_list
appended 10133 to zip_list
appended 10138 to zip_list
appended 10149 to zip_list
appended 10150 to zip_list
appended 10151 to zip_list
appended 10152 to zip_list
appended 10153 to zip_list
appended 10154 to zip_list
appended 10155 to zip_list
appended 10156 to zip_list
appended 10157 to zip_list
appended 10158 to zip_list
appended 10159 to zip_list
appended 10160 to zip_list
appended 10161 to zip_list
appended 10162 to zip_list
appended 10163 to zip_list
appended 10164 to zip_list
appended 10165 to zip_list
appended 10166 to zip_list
appended 10167 to zip_list
appended 10168 to zip_list
appended 10169 to zip_list
appended 10170 to zip_list
appended 10171 to zip_list
appended 10172 to zip_list
appended 10173 to zip_list
appended 10174 to zip_list
appended 10175 to zip_list
appended 10176 to zip_list
a

appended 12416 to zip_list
appended 12417 to zip_list
appended 12418 to zip_list
appended 12419 to zip_list
appended 12420 to zip_list
appended 12421 to zip_list
appended 12422 to zip_list
appended 12423 to zip_list
appended 12424 to zip_list
appended 12427 to zip_list
appended 12428 to zip_list
appended 12429 to zip_list
appended 12430 to zip_list
appended 12431 to zip_list
appended 12432 to zip_list
appended 12433 to zip_list
appended 12434 to zip_list
appended 12435 to zip_list
appended 12436 to zip_list
appended 12438 to zip_list
appended 12439 to zip_list
appended 12440 to zip_list
appended 12441 to zip_list
appended 12442 to zip_list
appended 12443 to zip_list
appended 12444 to zip_list
appended 12446 to zip_list
appended 12448 to zip_list
appended 12449 to zip_list
appended 12450 to zip_list
appended 12451 to zip_list
appended 12452 to zip_list
appended 12453 to zip_list
appended 12454 to zip_list
appended 12455 to zip_list
appended 12456 to zip_list
appended 12457 to zip_list
a

...

appended 90242 to zip_list
appended 90245 to zip_list
appended 90247 to zip_list
appended 90248 to zip_list
appended 90249 to zip_list
appended 90250 to zip_list
appended 90251 to zip_list
appended 90254 to zip_list
appended 90255 to zip_list
appended 90260 to zip_list
appended 90261 to zip_list
appended 90262 to zip_list
appended 90263 to zip_list
appended 90264 to zip_list
appended 90265 to zip_list
appended 90266 to zip_list
appended 90267 to zip_list
appended 90270 to zip_list
appended 90272 to zip_list
appended 90274 to zip_list
appended 90275 to zip_list
appended 90277 to zip_list
appended 90278 to zip_list
appended 90280 to zip_list
appended 90290 to zip_list
appended 90291 to zip_list
appended 90292 to zip_list
appended 90293 to zip_list
appended 90294 to zip_list
appended 90295 to zip_list
appended 90296 to zip_list
appended 90301 to zip_list
appended 90302 to zip_list
appended 90303 to zip_list
appended 90304 to zip_list
appended 90305 to zip_list
appended 90306 to zip_list
a

appended 92815 to zip_list
appended 92816 to zip_list
appended 92817 to zip_list
appended 92821 to zip_list
appended 92822 to zip_list
appended 92823 to zip_list
appended 92825 to zip_list
appended 92831 to zip_list
appended 92832 to zip_list
appended 92833 to zip_list
appended 92834 to zip_list
appended 92835 to zip_list
appended 92836 to zip_list
appended 92837 to zip_list
appended 92838 to zip_list
appended 92840 to zip_list
appended 92841 to zip_list
appended 92842 to zip_list
appended 92843 to zip_list
appended 92844 to zip_list
appended 92845 to zip_list
appended 92846 to zip_list
appended 92850 to zip_list
appended 92856 to zip_list
appended 92857 to zip_list
appended 92859 to zip_list
appended 92860 to zip_list
appended 92861 to zip_list
appended 92862 to zip_list
appended 92863 to zip_list
appended 92864 to zip_list
appended 92865 to zip_list
appended 92866 to zip_list
appended 92867 to zip_list
appended 92868 to zip_list
appended 92869 to zip_list
appended 92870 to zip_list
a

appended 94927 to zip_list
appended 94928 to zip_list
appended 94929 to zip_list
appended 94930 to zip_list
appended 94931 to zip_list
appended 94933 to zip_list
appended 94937 to zip_list
appended 94938 to zip_list
appended 94939 to zip_list
appended 94940 to zip_list
appended 94941 to zip_list
appended 94942 to zip_list
appended 94945 to zip_list
appended 94946 to zip_list
appended 94947 to zip_list
appended 94948 to zip_list
appended 94949 to zip_list
appended 94950 to zip_list
appended 94951 to zip_list
appended 94952 to zip_list
appended 94953 to zip_list
appended 94954 to zip_list
appended 94955 to zip_list
appended 94956 to zip_list
appended 94957 to zip_list
appended 94960 to zip_list
appended 94963 to zip_list
appended 94964 to zip_list
appended 94965 to zip_list
appended 94966 to zip_list
appended 94970 to zip_list
appended 94971 to zip_list
appended 94972 to zip_list
appended 94973 to zip_list
appended 94974 to zip_list
appended 94975 to zip_list
appended 94976 to zip_list
a

appended 97903 to zip_list
appended 97904 to zip_list
appended 97905 to zip_list
appended 97906 to zip_list
appended 97907 to zip_list
appended 97908 to zip_list
appended 97909 to zip_list
appended 97910 to zip_list
appended 97911 to zip_list
appended 97913 to zip_list
appended 97914 to zip_list
appended 97917 to zip_list
appended 97918 to zip_list
appended 97920 to zip_list
appended 98001 to zip_list
appended 98002 to zip_list
appended 98003 to zip_list
appended 98004 to zip_list
appended 98005 to zip_list
appended 98006 to zip_list
appended 98007 to zip_list
appended 98008 to zip_list
appended 98009 to zip_list
appended 98010 to zip_list
appended 98011 to zip_list
appended 98012 to zip_list
appended 98013 to zip_list
appended 98014 to zip_list
appended 98015 to zip_list
appended 98019 to zip_list
appended 98020 to zip_list
appended 98021 to zip_list
appended 98022 to zip_list
appended 98023 to zip_list
appended 98024 to zip_list
appended 98025 to zip_list
appended 98026 to zip_list
a

appended 99830 to zip_list
appended 99832 to zip_list
appended 99833 to zip_list
appended 99835 to zip_list
appended 99836 to zip_list
appended 99840 to zip_list
appended 99841 to zip_list
appended 99850 to zip_list
appended 99901 to zip_list
appended 99903 to zip_list
appended 99918 to zip_list
appended 99919 to zip_list
appended 99921 to zip_list
appended 99922 to zip_list
appended 99923 to zip_list
appended 99925 to zip_list
appended 99926 to zip_list
appended 99927 to zip_list
appended 99928 to zip_list
appended 99929 to zip_list
appended 99950 to zip_list


In [6]:
from socket import timeout

for zipp in zip_list:
    try:
        print('try setting loation for ', zipp)
        set_location(zipp)
        print(f'successfully set location for {zipp}')
        successes+=1
#         write_list_to_file(zip_list, 'resources/success_zips.csv')
    except NotFoundError:
        print("except", f'NotFoundError with zipcode {zipp}')
#         write_list_to_file(zip_list, 'resources/exception_zips.csv')
        exceptions+=1
        pass
print(f'successes = {successes}; exceptions = {exceptions}')

try setting loation for  501
except NotFoundError with zipcode 501
try setting loation for  544
except NotFoundError with zipcode 544
try setting loation for  601
except NotFoundError with zipcode 601
try setting loation for  602
except NotFoundError with zipcode 602
try setting loation for  603
except NotFoundError with zipcode 603
try setting loation for  604
except NotFoundError with zipcode 604
try setting loation for  605
except NotFoundError with zipcode 605
try setting loation for  606
except NotFoundError with zipcode 606
try setting loation for  610
except NotFoundError with zipcode 610
try setting loation for  611
except NotFoundError with zipcode 611
try setting loation for  612
except NotFoundError with zipcode 612
try setting loation for  613
except NotFoundError with zipcode 613
try setting loation for  614
except NotFoundError with zipcode 614
try setting loation for  616
except NotFoundError with zipcode 616
try setting loation for  617
except NotFoundError with zipcode

except NotFoundError with zipcode 851
try setting loation for  901
except NotFoundError with zipcode 901
try setting loation for  902
except NotFoundError with zipcode 902
try setting loation for  906
except NotFoundError with zipcode 906
try setting loation for  907
except NotFoundError with zipcode 907
try setting loation for  908
except NotFoundError with zipcode 908
try setting loation for  909
except NotFoundError with zipcode 909
try setting loation for  910
except NotFoundError with zipcode 910
try setting loation for  911
except NotFoundError with zipcode 911
try setting loation for  912
except NotFoundError with zipcode 912
try setting loation for  913
except NotFoundError with zipcode 913
try setting loation for  914
except NotFoundError with zipcode 914
try setting loation for  915
except NotFoundError with zipcode 915
try setting loation for  916
except NotFoundError with zipcode 916
try setting loation for  917
except NotFoundError with zipcode 917
try setting loation for 

except NotFoundError with zipcode 1080
try setting loation for  1081
except NotFoundError with zipcode 1081
try setting loation for  1082
except NotFoundError with zipcode 1082
try setting loation for  1083
except NotFoundError with zipcode 1083
try setting loation for  1084
except NotFoundError with zipcode 1084
try setting loation for  1085
except NotFoundError with zipcode 1085
try setting loation for  1086
except NotFoundError with zipcode 1086
try setting loation for  1088
except NotFoundError with zipcode 1088
try setting loation for  1089
except NotFoundError with zipcode 1089
try setting loation for  1090
except NotFoundError with zipcode 1090
try setting loation for  1092
except NotFoundError with zipcode 1092
try setting loation for  1093
except NotFoundError with zipcode 1093
try setting loation for  1094
except NotFoundError with zipcode 1094
try setting loation for  1095
except NotFoundError with zipcode 1095
try setting loation for  1096
except NotFoundError with zipcode 

except NotFoundError with zipcode 1452
try setting loation for  1453
except NotFoundError with zipcode 1453
try setting loation for  1460
except NotFoundError with zipcode 1460
try setting loation for  1462
except NotFoundError with zipcode 1462
try setting loation for  1463
except NotFoundError with zipcode 1463
try setting loation for  1464
except NotFoundError with zipcode 1464
try setting loation for  1467
except NotFoundError with zipcode 1467
try setting loation for  1468
except NotFoundError with zipcode 1468
try setting loation for  1469
except NotFoundError with zipcode 1469
try setting loation for  1470
except NotFoundError with zipcode 1470
try setting loation for  1471
except NotFoundError with zipcode 1471
try setting loation for  1472
except NotFoundError with zipcode 1472
try setting loation for  1473
except NotFoundError with zipcode 1473
try setting loation for  1474
except NotFoundError with zipcode 1474
try setting loation for  1475
except NotFoundError with zipcode 

except NotFoundError with zipcode 1803
try setting loation for  1805
except NotFoundError with zipcode 1805
try setting loation for  1806
except NotFoundError with zipcode 1806
try setting loation for  1807
except NotFoundError with zipcode 1807
try setting loation for  1808
except NotFoundError with zipcode 1808
try setting loation for  1810
except NotFoundError with zipcode 1810
try setting loation for  1812
except NotFoundError with zipcode 1812
try setting loation for  1813
except NotFoundError with zipcode 1813
try setting loation for  1815
except NotFoundError with zipcode 1815
try setting loation for  1821
except NotFoundError with zipcode 1821
try setting loation for  1822
except NotFoundError with zipcode 1822
try setting loation for  1824
except NotFoundError with zipcode 1824
try setting loation for  1826
except NotFoundError with zipcode 1826
try setting loation for  1827
except NotFoundError with zipcode 1827
try setting loation for  1830
except NotFoundError with zipcode 

...

except NotFoundError with zipcode 9305
try setting loation for  9306
except NotFoundError with zipcode 9306
try setting loation for  9307
except NotFoundError with zipcode 9307
try setting loation for  9308
except NotFoundError with zipcode 9308
try setting loation for  9309
except NotFoundError with zipcode 9309
try setting loation for  9310
except NotFoundError with zipcode 9310
try setting loation for  9311
except NotFoundError with zipcode 9311
try setting loation for  9312
except NotFoundError with zipcode 9312
try setting loation for  9313
except NotFoundError with zipcode 9313
try setting loation for  9314
except NotFoundError with zipcode 9314
try setting loation for  9315
except NotFoundError with zipcode 9315
try setting loation for  9316
except NotFoundError with zipcode 9316
try setting loation for  9317
except NotFoundError with zipcode 9317
try setting loation for  9318
except NotFoundError with zipcode 9318
try setting loation for  9319
except NotFoundError with zipcode 

except NotFoundError with zipcode 9524
try setting loation for  9532
except NotFoundError with zipcode 9532
try setting loation for  9534
except NotFoundError with zipcode 9534
try setting loation for  9543
except NotFoundError with zipcode 9543
try setting loation for  9545
except NotFoundError with zipcode 9545
try setting loation for  9549
except NotFoundError with zipcode 9549
try setting loation for  9550
except NotFoundError with zipcode 9550
try setting loation for  9554
except NotFoundError with zipcode 9554
try setting loation for  9556
except NotFoundError with zipcode 9556
try setting loation for  9557
except NotFoundError with zipcode 9557
try setting loation for  9564
except NotFoundError with zipcode 9564
try setting loation for  9565
except NotFoundError with zipcode 9565
try setting loation for  9566
except NotFoundError with zipcode 9566
try setting loation for  9567
except NotFoundError with zipcode 9567
try setting loation for  9568
except NotFoundError with zipcode 

except NotFoundError with zipcode 9745
try setting loation for  9746
except NotFoundError with zipcode 9746
try setting loation for  9747
except NotFoundError with zipcode 9747
try setting loation for  9748
except NotFoundError with zipcode 9748
try setting loation for  9749
except NotFoundError with zipcode 9749
try setting loation for  9750
except NotFoundError with zipcode 9750
try setting loation for  9751
except NotFoundError with zipcode 9751
try setting loation for  9752
except NotFoundError with zipcode 9752
try setting loation for  9753
except NotFoundError with zipcode 9753
try setting loation for  9754
except NotFoundError with zipcode 9754
try setting loation for  9755
except NotFoundError with zipcode 9755
try setting loation for  9756
except NotFoundError with zipcode 9756
try setting loation for  9757
except NotFoundError with zipcode 9757
try setting loation for  9758
except NotFoundError with zipcode 9758
try setting loation for  9759
except NotFoundError with zipcode 

successfully set location for 10022
try setting loation for  10023
successfully set location for 10023
try setting loation for  10024
successfully set location for 10024
try setting loation for  10025
successfully set location for 10025
try setting loation for  10026
successfully set location for 10026
try setting loation for  10027
successfully set location for 10027
try setting loation for  10028
successfully set location for 10028
try setting loation for  10029
successfully set location for 10029
try setting loation for  10030
successfully set location for 10030
try setting loation for  10031
successfully set location for 10031
try setting loation for  10032
successfully set location for 10032
try setting loation for  10033
successfully set location for 10033
try setting loation for  10034
successfully set location for 10034
try setting loation for  10035
successfully set location for 10035
try setting loation for  10036
successfully set location for 10036
try setting loation for  1

APICallTimeoutError: Exception in calling OWM Weather API.
Reason: API call timeouted
Caused by: None